In [1]:
import csv

<h2> I start with importing the csv files for the list of hexamers, list of 16nt sequences, and a 1:1 corresponding list of LEIs<h2>

In [8]:
datafile = open('./data/hexamer_array.csv', 'r')
datareader = csv.reader(datafile, delimiter=',')
hexamers = []
for row in datareader:
    hexamers.append(row[0])
print (hexamers[0])
datafile.close()

datafile = open('./data/all_sequences.csv', 'r')
datareader = csv.reader(datafile, delimiter=',')
sequences = []
for row in datareader:
    sequences.append(row)
print (sequences[0:2][0:5])
datafile.close()

datafile = open('./data/lei_values.csv', 'r')
datareader = csv.reader(datafile, delimiter=',')
leis = []
for row in datareader:
    leis.append(row)
print (leis[0:2][0:5])
datafile.close()

datafile = open('./data/neutral_motifs.csv', 'r')
datareader = csv.reader(datafile, delimiter=',')
n_motifs = []
for row in datareader:
    n_motifs.append(row[0])
print (n_motifs[0])
datafile.close()

# for sequences and leis, the first set of pointers point to the row, which there are 5 locations
# In order, the columns correspond to WA, WD, HA, HM, HD
# the second row points to the mutation, which there are 4096 for each variant

AAAAAA
[['GAGTTAAAAAAGGGAG', 'AATGGAAAAAAGGGCA', 'TACCCAAAAAACCAGA', 'TGGCAAAAAAATGCTC', 'CTGTGAAAAAATGCAC'], ['GAGTTAAAAACGGGAG', 'AATGGAAAAACGGGCA', 'TACCCAAAAACCCAGA', 'TGGCAAAAAACTGCTC', 'CTGTGAAAAACTGCAC']]
[['-0.371', '-0.101', '-0.464', '-0.8019', '0.0889'], ['-0.0103', '0.7433', '-1.4827', '0.633', '0.4231']]
AAAAAA


<h3>Algorithm for finding all instances of each hexamer and averages the LEI values<h3>
<p>this will produce a dictionary with hexamers as keys and average LEI values as values<p>

In [45]:
# lei_vals = []
# for x in range(0,5):
#     for y in range(0,4096):
#         if sequences[y][x].find("GACGTC") != -1:
#             lei_vals.append([float(leis[y][x]),6- abs(5-sequences[y][x].find("GACGTC"))])
# print ([i[1] for i in lei_vals])
# total_weight = sum([i[1] for i in lei_vals])
# print ([i[0]*i[1] for i in lei_vals])
# weighted_sum = sum([i[0]*i[1] for i in lei_vals])
# avg = weighted_sum/total_weight
# avg_of_rest = (752.923 - sum([i[0] for i in lei_vals]))/(4096*5 - len(lei_vals))
# print(len(lei_vals))
# ESE = avg - avg_of_rest
# print (ESE)

[6, 5, 5, 5, 5, 6, 5, 5, 6, 5, 5, 6, 5, 5, 5, 5, 6]
[6.1968, 4.035, 5.913500000000001, 6.383, 5.1034999999999995, 6.477599999999999, 4.3185, 5.6765, 5.3724, 6.538, 4.543, 4.969200000000001, 4.081, 5.3735, 5.4955, 3.6420000000000003, 7.6428]
17
0.9836281089445988


In [24]:
# hex_dic = dict.fromkeys(hexamers, 0) #0 is the default value before we add in the averaged LEI values
# lei_vals = []

# for hexamer in hexamers:
#     for x in range(0,5):
#         for y in range(0,4096):
#             if sequences[y][x].find(hexamer) != -1:
#                 lei_vals.append(float(leis[y][x]))
#     avg = sum(lei_vals)/len(lei_vals)
#     avg_rest = (752.923 - sum(lei_vals))/(4096*5 - len(lei_vals))
#     hex_dic[hexamer] = [avg-avg_rest, len(lei_vals)]
#     lei_vals = []


In [5]:
# print (hex_dic["CTCAAA"]) # this allows for testing

I will leave out the p test for now, I will just copy the data for significance from the excel and apply it to the weighted recalculation

In [9]:
#weighted recalculation

hex_dic = dict.fromkeys(hexamers, 0) #0 is the default value before we add in the averaged LEI values
lei_vals = []

for hexamer in hexamers:
    for x in range(0,5):
        for y in range(0,4096):
            if sequences[y][x].find(hexamer) != -1:
                lei_vals.append([float(leis[y][x]),6- abs(5-sequences[y][x].find(hexamer))])
    total_weight = sum([i[1] for i in lei_vals])
    weighted_sum = sum([i[0]*i[1] for i in lei_vals])
    weighted_avg = weighted_sum/total_weight
    avg_rest = (752.923 - sum([i[0] for i in lei_vals]))/(4096*5 - len(lei_vals))
    if weighted_avg-avg_rest > 0:
        hex_dic[hexamer] = [weighted_avg-avg_rest, len(lei_vals), 1]
    else:
        hex_dic[hexamer] = [weighted_avg-avg_rest, len(lei_vals), -1]
    lei_vals = []

In [11]:
for neutral_motif in n_motifs:
    hex_dic[neutral_motif][2] = 0

In [14]:
print (hex_dic["AAAAAA"]) # this allows for testing

[-0.16719185672670428, 8, 0]


In [21]:
with open("./data/weighted_hex_scores.csv", 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['hex','score','count','assignment'])
    for data in hex_dic:
        writer.writerow([data,hex_dic[data][0],hex_dic[data][1],hex_dic[data][2]])